In [11]:
### import and initialize modules ###
%load_ext autoreload
%autoreload 2

%matplotlib inline
# ! if you want to use this, run  'conda install -c conda-forge scalene' for your conda env
#%load_ext scalene 
# ! Or 'conda install -c conda-forge line_profiler' for this one.
%load_ext line_profiler
import sys
sys.path.append("../code")
import numpy as np
import pandas as pd 
from registration_project import rigid_reg_cc_demo, affine_reg_cc_demo, affine_reg_mi_demo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
LOADING
Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version. To profile in line mode, use
`%scrun [options] statement`. To profile in cell mode, use `%%scalene
[options]` followed by your code.


In [12]:
### import and read data ###
path = 'source\data\image_data'



In [13]:
### Registration ###
# We can proba
%lprun -f affine_reg_cc_demo affine_reg_cc_demo(r"../data/image_data/1_1_t1.tif",r"../data/image_data/1_1_t1_d.tif",False) # example, but not applicable for current usage
res = %timeit -o rigid_reg_cc_demo('../data/image_data/1_1_t1.tif','../data/image_data/1_1_t1_d.tif',False) # for the main registration problem

Final similarity of ca. 0.9299874408059998


Timer unit: 1e-07 s

Total time: 22.907 s
File: c:\Users\quint\Downloads\assignments\project-imgreg-group19\source\reader\../code\registration_project.py
Function: affine_reg_cc_demo at line 104

Line #      Hits         Time  Per Hit   % Time  Line Contents
   104                                           def affine_reg_cc_demo(img1,img2,imshow=False,num_iter=200, learning_rate=0.001):
   105                                           
   106                                               # read the fixed and moving images
   107                                               # change these in order to read different images
   108         1      26251.0  26251.0      0.0      I = plt.imread(img1)
   109         1      28586.0  28586.0      0.0      Im = plt.imread(img2)
   110                                           
   111                                               # initial values for the parameters
   112                                               # we start with the identity 

In [ ]:
### Statistical analysis ###

# input: list of computation times per type of registration: time_rigid, time_affine, time_rigid_all, time_affine_all
# input: list of computation times per formula: rigid_cc_time, affine_cc_time, affine_mi_time
# input: list of affine transformation matrices per registration: transformations (rot, scalex, scaley, shearx, sheary, translatex, translatey)

# output: significance testing between the types of registration
# output: power analysis of significance testing
# output: Boxplots for the computation time data spread


aggr_rigid_affine = time_rigid + time_affine # List containing all tests, containing the time using our chosen method
data = [np.transpose(time_rigid),np.transpose(time_affine),np.transpose(aggr_rigid_affine),np.transpose(time_rigid_all),np.transpose(time_affine_all)]
df_time = pd.DataFrame(data,columns=['T_rigid',
                                     'T_affine',
                                     'T_aggr_all'
                                     'T_rigid_all',
                                     'T_affine_all'])
box = df_time.boxplot(grid=True)
box.plot(figsize=(1,3))

# Test for equal variance
print("Variance of comp. time for rigid transformations: ",np.var(a=df_time['T_rigid'])
print("Variance of comp. time for affine transformations: ",np.var(a=df_time['T_affine'])

# Variance vary less than factor 4:
    # Test if the computation time to use affine transformation compared to rigid transformation is longer
print("t test affine_all>rigid_all: ",stats.ttest_ind(a=df_time['T_affine_all'],b=df_time['T_rigid_all'], equal_var=True,alternative='greater')) 

    # Test if the aggregated computation time is shorter than all affine registration  
print("t test affine_all>aggr: ",stats.ttest_ind(a=df_time['T_affine_all'],b=df_time['T_aggr_all'], equal_var=True,alternative='greater'))

# See if our decision program is working well
rigid_decision = [1 if time_rigid[i].isnumeric() for i in range(len(time_rigid)) else 0] # Records if rigid was chosen
# transform_rigid = transformations[rigid_decision] # affine transformation matrices for which we used rigid
# transform_affine = transformations[~rigid_decision] # affine transformation matrices for which we used affine
data2 = [rigid_decision]
sum_affine = transformations[:][1]+transformations[:][2]+transformations[:][3]+transformations[:][4]
df_transform = pd.DataFrame({"Decision": rigid_decision,
                             "Sum_affine": sum_affine})
box2 = df_transform.plot.box(column="Sum_affine",by="Decision",figsize=(5,5))

# Basic statistics of the average shear and scaling present 
    # Shear
print("Mean of all shear: ",mean(transformations[:][1:2]))
print("Mean of shear of rigid: ", mean(transformation[rigid_decision][1:2]))
print("Mean of shear of affine: ", mean(transformation[rigid_decision][1:2]))

print("Variance of all shear: ",var(transformations[:][1:2]))
print("Variance of shear of rigid: ", var(transformation[rigid_decision][1:2]))
print("Variance of shear of affine: ", var(transformation[rigid_decision][1:2]))

    #Scaling
print("Mean of all scaling: ",mean(transformations[:][3:4]))
print("Mean of scaling of rigid: ", mean(transformation[~rigid_decision][3:4]))
print("Mean of scaling of affine: ", mean(transformation[~rigid_decision][3:4]))

print("Variance of all scaling: ",var(transformations[:][3:4]))
print("Variance of scaling of rigid: ", var(transformation[~rigid_decision][3:4]))
print("Variance of scaling of affine: ", var(transformation[~rigid_decision][3:4]))

